In [5]:
from osgeo import gdal, gdalnumeric, ogr, osr
import os, sys
import math
import operator
gdal.UseExceptions()

def CLIP( shapefile_path, raster_path, outputName ):
    # Load the source data as a gdalnumeric array
    #shapefile_path = "C:/Users/Ray/Desktop/Clip/studyArea.shp"
    shapefile_path = str(shapefile_path)
    raster_path = str(raster_path)
    #raster_path = "C:/Users/Ray/Desktop/Clip/1.tif"
    print (shapefile_path)
    print (raster_path)
    srcArray = gdalnumeric.LoadFile(raster_path)

    # Also load as a gdal image to get geotransform
    # (world file) info
    srcImage = gdal.Open(raster_path)
    geoTrans = srcImage.GetGeoTransform()

    # Create an OGR layer from a boundary shapefile
    print( shapefile_path)

    shapef = ogr.Open(shapefile_path)
    print (type(shapef))
    lyr = shapef.GetLayer( os.path.split( os.path.splitext( shapefile_path )[0] )[1] )
    poly = lyr.GetNextFeature()

    # Convert the layer extent to image pixel coordinates
    minX, maxX, minY, maxY = lyr.GetExtent()
    ulX, ulY = world2Pixel(geoTrans, minX, maxY)
    lrX, lrY = world2Pixel(geoTrans, maxX, minY)

    # Calculate the pixel size of the new image
    pxWidth = int(lrX - ulX)
    pxHeight = int(lrY - ulY)

    clip = srcArray[ ulY:lrY, ulX:lrX]

    #
    # EDIT: create pixel offset to pass to new image Projection info
    #
    xoffset =  ulX
    yoffset =  ulY
    print( "Xoffset, Yoffset = ( %f, %f )" % ( xoffset, yoffset ))
    
    # Create a new geomatrix for the image
    geoTrans = list(geoTrans)
    geoTrans[0] = minX
    geoTrans[3] = maxY

    # Map points to pixels for drawing the
    # boundary on a blank 8-bit,
    # black and white, mask image.
    points = []
    pixels = []
    geom = poly.GetGeometryRef()
    pts = geom.GetGeometryRef(0)
    for p in range(pts.GetPointCount()):
      points.append((pts.GetX(p), pts.GetY(p)))
    for p in points:
      pixels.append(world2Pixel(geoTrans, p[0], p[1]))
    rasterPoly = Image.new("L", (pxWidth, pxHeight), 1)
    rasterize = ImageDraw.Draw(rasterPoly)
    rasterize.polygon(pixels, 0)
    mask = imageToArray(rasterPoly)

    # Clip the image using the mask
    clip = gdalnumeric.choose(mask, \
        (clip, 0)).astype('float64')


    # Save new tiff
    #
    #  EDIT: instead of SaveArray, let's break all the
    #  SaveArray steps out more explicity so
    #  we can overwrite the offset of the destination
    #  raster
    #
    ### the old way using SaveArray
    #
    # gdalnumeric.SaveArray(clip, "OUTPUT.tif", format="GTiff", prototype=raster_path)
    #
    ###
    #
    gtiffDriver = gdal.GetDriverByName( 'GTiff' )
    if gtiffDriver is None:
        raise ValueError("Can't find GeoTiff Driver")
    gtiffDriver.CreateCopy( outputName,
        OpenArray( clip, prototype_ds=raster_path, xoff=xoffset, yoff=yoffset )
    )

    # Save as an 8-bit jpeg for an easy, quick preview
    clip = clip.astype('float64')
    
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install gdal


ModuleNotFoundError: No module named 'osgeo'

In [6]:
import requests
import json
import getpass
import os
import tarfile
from urllib.parse import urlparse
import shutil
from osgeo import gdal


ModuleNotFoundError: No module named 'osgeo'

In [2]:
def get_apikey(username, password):
    payload = dict(username=username, password=password, catalogId='EE', authType='EROS')
    data = dict(jsonRequest=json.dumps(payload))
    r = requests.post('https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/login', data=data)
    creds = r.json()
    apiKey = creds['data']
    return apiKey


In [3]:
username = 'skylinegis'
password = 'pepsiav123pepsiav123'

In [4]:
apiKey = get_apikey(username, password)

In [6]:

def get_scenes(region, horizontal, vertical):
    myfilter = {
            "filterType": "and", 
            "childFilters": [
                {"filterType":"value","fieldId":21789,"value":region}, #Grid Region
                {"filterType":"value","fieldId":21787,"value":horizontal}, #Horizontal
                {"filterType":"value","fieldId":21788,"value":vertical}, #Vertical
            ]
        }

    request_code = 'search'
    baseurl = f'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/{request_code}'
    payload = {'apiKey':apiKey, 
               'datasetName': 'ARD_TILE', 
               'additionalCriteria': myfilter,  
               'maxResults':10000}
    data = dict(jsonRequest=json.dumps(payload))
    r = requests.get(baseurl, params=data)
    response = r.json()
    print(response)
    return response

In [ ]:
lat = '-105.05487442016603'
lon = '40.35345235'
converturl = 'https://landsatlook.usgs.gov/arcgis/rest/services/LLook_Outlines/MapServer/6/query?where=1%3D1&outSr=4326&outFields=H,V&inSr=4326&geometry=%7B%22x%22%3A' + lat + '%2C%22y%22%3A' + lon + '%2C%22spatialReference%22%3A%7B%22wkid%22%3A4326%7D%7D&geometryType=esriGeometryPoint&spatialRel=esriSpatialRelIntersects&orderByFields=H%20ASC&f=json'
print(converturl)
r = requests.get(converturl)
# Query USGS Earth Explorer API for scenes
region='CU'
horizontal=h
vertical=v

In [10]:
response = get_scenes(region, horizontal, vertical)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:

print(response)
print('Found {} scenes'.format(response['data']['totalHits']))
print('Returned {} scenes'.format(response['data']['numberReturned']))
print('First acquisition: ', response['data']['results'][0]['acquisitionDate'])
print('Last acquisition: ', response['data']['results'][-1]['acquisitionDate'])


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

